## Module 3 - Dans ce notebook jupyter, les indicateurs de perfomance sont calculés 
* Etape 3a - Réglage: Importer les modules/librairies, importer les données, créer un dossier de sortie
* Etape 3b - Calculer l'uniformité
* Etape 3c - Calculer l'efficité (fraction bénéfique)
* Etape 3d - Calculer l'adquation
* Etape 3e - Calculer le déficit hydrique relatif 
**=====================================================================================================================**

![title](img/Fig3_1.png)

**=====================================================================================================================**
#### <span style='background :lightgreen' > References:
* Karimi, P., Bongani, B., Blatchford, M., and de Fraiture, C.: Global satellite-based ET products for the local level irrigation management: An application of irrigation performance assessment in the sugarbelt of Swaziland, Remote Sensing, 11, 705, 2019.
* Bastiaanssen, W. G., and Bos, M.: Irrigation performance indicators based on remotely sensed data: a review of literature, Irrigation and drainage systems, 13, 291-311, 1999.
* Bastiaanssen, W. G., Van der Wal, T., and Visser, T.: Diagnosis of regional evaporation by remote sensing to support irrigation performance assessment, Irrigation and Drainage Systems, 10, 1-23, 1996.

## Etape 3a - Réglage

## i) Importer les packages/librairies

In [1]:
import os
import sys
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# changer le répertoire à là ou les modules sont enregistrés
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## ii) Importer les données d'entrée
*  l'T, AETI (ETRI), ETp saisonnière

In [2]:
# obtenir les données saisonnières
T_fhs    = glob.glob(r'..\Data\2L2_T_season\*.tif')
AETI_fhs = glob.glob(r'..\Data\2L2_AETI_season\*.tif')
ETp_fhs  = glob.glob(r'..\Data\2L1_ETp_season\*.tif')

## iii) Dossier de sortie: Créer un ou connecter l'existant

In [3]:
# output_folder (dossier de sortie)
output_folderBF = r'..\Data\3_BeneficialFraction'  # créer un dossier de sortie
output_folderAd = r'..\Data\3_Adequacy'  # créer un dossier de sortie

# En créer un si le dossier n'existe pas
if not os.path.exists(output_folderBF):
    os.makedirs(output_folderBF)
if not os.path.exists(output_folderAd):
    os.makedirs(output_folderAd)

## Etape 3b - Calculer l'uniformité de la consommation de l'eau
* Equité est définie comme étant le coéfficient de variation (cv) de l'ET réelle dans la zone d'intérêt.
* Elle mesure l'égalité de l'apport d'eau dans un périmètre irrigué. 
* Remarque: un CV of 0 à 10% est bon, 10 à 25% est acceptable et un CV > 25% est faible comme uniformité (Bastiaanssen et al., 1996) 
<br/>

In [4]:
# Uniformité de la consommation d'eau 
# Obtenir l'AETI (ETRI) saisonnière
# AETI_fhs = glob.glob(r'..\Data\2L2_AETI_season\*.tif') 

for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    AETIm   = np.nanmean(AETI)
    AETIsd  = np.nanstd(AETI)
    
    CV_AETI = (AETIsd / AETIm) * 100
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    if CV_AETI < 10:
        U = 'Good Uniformity'
    if (CV_AETI >= 10) and (CV_AETI < 25):
        U = 'Fair Uniformity'
    else: 
        U = 'Poor Uniformity'

    print ('CV of AETI in', date, '=', round(CV_AETI, 1), ',', U)

## Etape 3c - Calculer l'efficacité (fraction bénéfique)
* La fraction bénéfique est le ratio de l'eau qui est consommée et transpirée comparée à la consommation totale d'eau dans le champs (ETa). 
* $Fraction bénéfique = \frac{T_a}{ET_a}$
* C'est une mesure de l'efficacité de l'eau agricole et des pratiques agricole dans l'utilisation de l'eau pour la production culturale.

In [5]:
# collecter les Geoinfo tels que la projection, les axes x et y
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster
  
for Tfh, ETfh in zip(T_fhs, AETI_fhs):
    T    = gis.OpenAsArray(Tfh,  nan_values=True)
    AETI = gis.OpenAsArray(ETfh, nan_values=True)
    
    T_over_AETI = T/AETI
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    # mettre à jour le nom du fichier, et enregistrer dans le dossier de sortie
    basename  = os.path.basename(ETfh).replace('AETI', 'BF')  
    output_fn = os.path.join(output_folderBF, basename)
    gis.CreateGeoTiff(output_fn, T_over_AETI, driver, NDV, xsize, ysize, GeoT, Projection) 
    
    # Tracer la carte du raster
    seasonal = T_over_AETI
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='BF [-]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Beneficial fraction [-] ' + date)
#     plt.clim(0,1)
    plt.show ()

## Etape 3d - Calculer  l'adequation (évapotranspiration relative)
$Adequation= \frac{ET_a}{ET_p}$

In [6]:
## Calcule et enregistre la couche raster de l'adéquation

# collecte des Geoinfo tels que la projection, les axes x et y
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster

for ETfh, ETpfh in zip(AETI_fhs, ETp_fhs):
    AETI = gis.OpenAsArray(ETfh,  nan_values=True)
    ETp  = gis.OpenAsArray(ETpfh, nan_values=True)
    
    ETa_by_ETp = AETI / ETp
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(ETfh).split('.')[0].replace('AETI', '').replace('_', ' ') 
        
    # mettre à jour le nom du fichier, et enregistrer dans le dossier de sortie
    basename  = os.path.basename(ETfh).replace('AETI', 'Adequacy')
    output_fn = os.path.join(output_folderAd, basename)
    gis.CreateGeoTiff(output_fn, ETa_by_ETp, driver, NDV, xsize, ysize, GeoT, Projection) 
  
    # Tracer la carte du raster
    seasonal = ETa_by_ETp
    
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='RdYlGn', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='Adequacy [-]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # ajouter les titres des axes
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('Adequacy [-] ' + date)
    plt.show ()
    ;

## Etape 3e - Calculer le Déficit hydrique de l'Eau (RWD)
$RWD= 1-\frac{ET_a}{ET_x}$
<br/>${ET_x} = $ Peut être ETp ou 99 centile de l'évapotranspiration réelle

In [7]:
# collecte des Geoinfo tels que la projection, les axes x et y
in_fh = AETI_fhs[0]      
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # Obtenir l'étendue spatiale du raster
   
for i in range(len(AETI_fhs)):
    AETI = gis.OpenAsArray(AETI_fhs[i], nan_values=True)
    
    # changer la forme (reshape)des matrices
    AETI1_1D  = np.reshape(AETI,  AETI.shape[0]*AETI.shape[1])
    ETx       = np.nanpercentile(AETI1_1D, 99)
    
    AETI_mean = np.nanmean(AETI)
   
    RWD = 1-(AETI_mean/ETx)
    
    # Identifier la date à partir du nom du fichier
    date  = os.path.basename(AETI_fhs[i]).split('.')[0].replace('AETI', '').replace('_', ' ') 
    
    print ('Relative water deficit', date, '=', round(RWD, 2))
    ;